## Data Cleaning

Now since we have extracted data from the website, it is not cleaned and ready to be analyzed yet. The reviews section will need to be cleaned for punctuations, spellings and other characters. 

In [1]:
#imports

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

#regex
import re

In [2]:
#get current working directory

cwd = os.getcwd()
#create a dataframe from csv file

df = pd.read_csv("/kaggle/input/airways-customer-data/airline_data.csv", index_col=0)

In [3]:
df.head()

,reviews,rates,date,country
0,✅ Trip Verified | Excellent service both on th...,\n\t\t\t\t\t\t\t\t\t\t\t\t\t5,24th January 2023,United Kingdom
1,✅ Trip Verified | Good lounge at Cape Town. O...,9,24th January 2023,United Kingdom
2,✅ Trip Verified | A really excellent journey....,9,23rd January 2023,United Kingdom
3,✅ Trip Verified | This flight was one of the ...,9,21st January 2023,United Kingdom
4,Not Verified | It seems that there is a race t...,2,19th January 2023,United States


We will also create a column which mentions if the user is verified or not. 

In [4]:
df['verified'] = df.reviews.str.contains("Trip Verified")

In [5]:
df['verified']

0        True
1        True
2        True
3        True
4       False
        ...  
3413    False
3414    False
3415    False
3416    False
3417    False
Name: verified, Length: 3418, dtype: bool

### Cleaning Reviews

We will extract the column of reviews into a separate dataframe and clean it for semantic analysis

In [6]:
#for lemmatization of words we will use nltk library
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
lemma = WordNetLemmatizer()


reviews_data = df.reviews.str.strip("✅ Trip Verified |")

#create an empty list to collect cleaned data corpus
comments =[]

#loop through each review, remove punctuations, small case it, join it and add it to corpus
for rev in reviews_data:
    rev = re.sub('[^a-zA-Z]',' ', rev)
    rev = rev.lower()
    rev = rev.split()
    rev = [lemma.lemmatize(word) for word in rev if word not in set(stopwords.words("english"))]
    rev = " ".join(rev)
    comments.append(rev)

In [7]:
# add the corpus to the original dataframe

df['comments'] = comments

In [8]:
df.head()

,reviews,rates,date,country,verified,comments
0,✅ Trip Verified | Excellent service both on th...,\n\t\t\t\t\t\t\t\t\t\t\t\t\t5,24th January 2023,United Kingdom,True,excellent service ground board first class pro...
1,✅ Trip Verified | Good lounge at Cape Town. O...,9,24th January 2023,United Kingdom,True,good lounge cape town time departure drink hig...
2,✅ Trip Verified | A really excellent journey....,9,23rd January 2023,United Kingdom,True,really excellent journey lounge overcrowded la...
3,✅ Trip Verified | This flight was one of the ...,9,21st January 2023,United Kingdom,True,flight one worst ever life wanted pamper bough...
4,Not Verified | It seems that there is a race t...,2,19th January 2023,United States,False,verified seems race bottom amongst airline can...


### Cleaning/Fromat date

In [9]:
df.dtypes

reviews     object
rates       object
date        object
country     object
verified      bool
comments    object
dtype: object

In [10]:
# convert the date to datetime format

df.date = pd.to_datetime(df.date)

In [11]:
df.date.head()

0   2023-01-24
1   2023-01-24
2   2023-01-23
3   2023-01-21
4   2023-01-19
Name: date, dtype: datetime64[ns]

### Cleaning ratings with stars

In [12]:
#check for unique values
df.rates.unique()

array(['\n\t\t\t\t\t\t\t\t\t\t\t\t\t5', '9', '2', '3', '1', '4', '5', '8',
       '6', '7', '10', 'None'], dtype=object)

In [13]:
# remove the \t and \n from the ratings
df.rates = df.rates.str.strip("\n\t\t\t\t\t\t\t\t\t\t\t\t\t")

In [14]:
df.rates.value_counts()

1       745
2       385
3       380
8       344
10      303
7       297
9       293
5       255
4       232
6       179
None      5
Name: rates, dtype: int64

There are 5 rows having values "None" in the ratings. We will drop all these 5 rows. 

In [15]:
# drop the rows where the value of ratings is None
df.drop(df[df.rates == "None"].index, axis=0, inplace=True)

In [16]:
#check the unique values again
df.rates.unique()

array(['5', '9', '2', '3', '1', '4', '8', '6', '7', '10'], dtype=object)

## Check for null Values

In [17]:
df.isnull().value_counts()

reviews  rates  date   country  verified  comments
False    False  False  False    False     False       3411
                       True     False     False          2
dtype: int64

In [18]:
df.country.isnull().value_counts()

False    3411
True        2
Name: country, dtype: int64

We have two missing values for country. For this we can just remove those two reviews (rows) from the dataframe. 

In [19]:
#drop the rows using index where the country value is null
df.drop(df[df.country.isnull() == True].index, axis=0, inplace=True)

In [20]:
df.shape

(3411, 6)

In [21]:
df.isnull().value_counts()

reviews  rates  date   country  verified  comments
False    False  False  False    False     False       3411
dtype: int64

In [22]:
#resetting the index
df.reset_index(drop=True)

,reviews,rates,date,country,verified,comments
0,✅ Trip Verified | Excellent service both on th...,5,2023-01-24,United Kingdom,True,excellent service ground board first class pro...
1,✅ Trip Verified | Good lounge at Cape Town. O...,9,2023-01-24,United Kingdom,True,good lounge cape town time departure drink hig...
2,✅ Trip Verified | A really excellent journey....,9,2023-01-23,United Kingdom,True,really excellent journey lounge overcrowded la...
3,✅ Trip Verified | This flight was one of the ...,9,2023-01-21,United Kingdom,True,flight one worst ever life wanted pamper bough...
4,Not Verified | It seems that there is a race t...,2,2023-01-19,United States,False,verified seems race bottom amongst airline can...
...,...,...,...,...,...,...
3406,Flew British Airways from Sydney to Venice on ...,6,2014-05-20,Australia,False,flew british airway sydney venice nd april ret...
3407,Recently for our honeymoon we took advantage o...,10,2014-05-19,United Kingdom,False,recently honeymoon took advantage ba promo fly...
3408,LGW-Antigua on 29 April and Antigua-LGW on 13 ...,1,2014-05-18,United Kingdom,False,lgw antigua april antigua lgw may boeing start...
3409,LHR-JTR-LHR. A tale of opposites for BA's new ...,4,2014-05-18,United Kingdom,False,lhr jtr lhr tale opposite ba new route santori...


*****

Now our data is all cleaned and ready for data visualization and data analysis.

In [23]:
# export the cleaned data

df.to_csv(cwd + "/cleaned-reviews.csv")